In [ ]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

In [ ]:
print(bucket)

In [ ]:
prefix = 'dogscats-images'

s3_train_path = 's3://{}/{}/input/train/'.format(bucket, prefix)
s3_val_path = 's3://{}/{}/input/val/'.format(bucket, prefix)
s3_train_lst_path = 's3://{}/{}/input/train_lst/'.format(bucket, prefix)
s3_val_lst_path = 's3://{}/{}/input/val_lst/'.format(bucket, prefix)

s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_train_path)
print(s3_val_path)
print(s3_train_lst_path)
print(s3_val_lst_path)
print(s3_output)

### Get the name of the image classification algorithm in our region

In [ ]:
from sagemaker.image_uris import retrieve

region = sess.boto_session.region_name    
container = retrieve('image-classification', region)
print(container)

### Configure the training job

In [ ]:
ic = sagemaker.estimator.Estimator(
    container,
    sagemaker.get_execution_role(), 
    instance_count=1, 
    instance_type='ml.p3.2xlarge',
    output_path=s3_output)

### Set algorithm parameters

In [ ]:
ic.set_hyperparameters(
    num_layers=18,               # Train a Resnet-18 model
    use_pretrained_model=0,      # Train from scratch
    num_classes=2,               # Dogs and cats
    num_training_samples=22500,  # Number of training samples
    mini_batch_size=128,
    resize=224,
    epochs=10)                   # Learn the training set 10 times

### Set dataset parameters

In [ ]:
from sagemaker import TrainingInput

train_data = TrainingInput(
    s3_train_path, 
    content_type='application/x-image')

val_data = TrainingInput(
    s3_val_path, 
    content_type='application/x-image')

train_lst_data = TrainingInput(
    s3_train_lst_path, 
    content_type='application/x-image')

val_lst_data = TrainingInput(
    s3_val_lst_path, 
    content_type='application/x-image')

s3_channels = {'train': train_data, 'validation': val_data, 
               'train_lst': train_lst_data, 'validation_lst': val_lst_data}

### Train the model

In [ ]:
ic.fit(inputs=s3_channels)

### Deploy the model

In [ ]:
ic_predictor = ic.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium')

### Download a test image

In [ ]:
# Dog
!wget -O /tmp/test.jpg https://upload.wikimedia.org/wikipedia/commons/b/b7/LabradorWeaving.jpg
file_name = '/tmp/test.jpg'
from IPython.display import Image
Image(file_name)

### Predict test image

In [ ]:
import boto3, json
import numpy as np

runtime = boto3.Session().client(service_name='runtime.sagemaker')

with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)
    
response = runtime.invoke_endpoint(EndpointName=ic_predictor.endpoint_name, 
                                   ContentType='application/x-image', 
                                   Body=payload)

result = response['Body'].read()
result = json.loads(result)
index = np.argmax(result)
print(result[index], index)

# You should see class index 1, corresponding to the 'dog' class

### Delete endpoint

In [ ]:
ic_predictor.delete_endpoint()